In [1]:
import pandas as pd

df = pd.read_csv('mobile_price_1.csv')

In [5]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
#speed - ordinal
#

In [13]:
df["resolution"] = df["px_height"] * df["px_width"] #3. Add a column that holds the total screen resolution for each device. Name it resolution

In [3]:
df["DPI_w"] = ((df['sc_h'] * df['sc_w']) * 0.393700787) / df['px_width'] #Converting the whole screen size in inches and then dividing it by the amount of pixel in width

In [11]:
print(df['memory'])

0        7.568
1       53.759
2       41.140
3       10.558
4       44.436
         ...  
1995     2.650
1996    39.573
1997    36.706
1998    46.218
1999    45.731
Name: memory, Length: 2000, dtype: float64


In [5]:
df["call_ratio"] = df["battery_power"] / df["talk_time"] # Add a column that holds the ratio battery_power/talk_time and name it call_ratio. 6.

In [10]:
df["memory"] = df["memory"]/1024 # Change the memory column to hold the memory in GB instead of MB.

In [ ]:
df.speed = df.speed.astype('category')
df.cores = df.cores.astype('category')
df.screen = df.screen.astype('category') # 8. Convert the following features into categorical series in the Dataframe: speed,screen,cores

In [3]:
df.describe()

,id,battery_power,m_dep,mobile_wt,px_height,px_width,ram,sc_h,sc_w,talk_time,gen,f_camera,camera,memory,price
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,1526.000000,1899.000000,2000.000000,2000.00000
mean,999.500000,1238.518500,0.501750,140.249000,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,3.283000,5.648100,10.443918,32546.259000,862.02435
std,577.494589,439.418206,0.288416,35.399655,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.824773,4.140169,5.763826,18161.462247,230.26348
min,0.000000,501.000000,0.100000,80.000000,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,2.000000,1.000000,1.000000,1954.000000,307.11000
25%,499.750000,851.750000,0.200000,109.000000,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,3.000000,2.000000,6.000000,16421.000000,698.46750
50%,999.500000,1226.000000,0.500000,141.000000,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,4.000000,5.000000,10.000000,32303.000000,837.63000
75%,1499.250000,1615.250000,0.800000,170.000000,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,4.000000,8.000000,15.000000,48340.250000,1006.71250
max,1999.000000,1998.000000,1.000000,200.000000,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,4.000000,19.000000,20.000000,64922.000000,1728.97000


In [ ]:
no_camera_records = df[df['camera'].isna() & df['f_camera'].isna()]

In [ ]:
len(no_camera_records)#1. How many phones do not have a camera at all (front or back)

In [14]:
data_frame_manipulated = df[((df['camera'] >= 1) | df['f_camera'] >= 1) & (df['sim'] == 'Single') & (df['resolution'] > 12)]
data_frame_manipulated['battery_power'].mean(skipna = True) # What is the average battery power for single-sim phones that have a camera or front camera with a higher resolution than 12 megapixels?

1256.2735135135135

In [15]:
df[df[(df['wifi'] == 'none') & (df['screen'] == 'Touch') & (df['mobile_wt'] > 145)]['price'].max() == df['price']]['id'] # What is the ID and price of the most expensive phone that has no wifi, a touch screen and weighs more than 145 grams?

770    770
Name: id, dtype: int64

In [3]:
Bluetooth_per_gen = df[['gen', 'bluetooth']].copy()

In [78]:
#Bluetooth_per_gen['ram'].quantile([0,0.25,0.50,0.75])

In [4]:
Bluetooth_per_gen['bluetooth'] = Bluetooth_per_gen['bluetooth'].replace(['Yes' , 'No'], [1 ,0])

NameError: name 'Bluetooth_per_gen' is not defined

In [6]:
Bluetooth_per_gen['bluetooth'].unique()

array([0, 1], dtype=int64)

In [44]:
Bluetooth_per_gen

,gen,bluetooth
0,2,0
1,4,1
2,4,1
3,3,1
4,4,1
...,...,...
1995,4,1
1996,3,1
1997,4,0
1998,4,0


In [9]:
Bluetooth_per_gen.pivot_table(index='gen', values='bluetooth', columns='bluetooth', aggfunc=sum)

bluetooth
gen
2
3
4


In [10]:
ram = pd.cut(df['ram'], [0,1000,2000,3000,4000]) # Create a pivot table that shows the percentage of phones with Bluetooth per generation,  pivoted around the phone generation and split by “ram” quartiles. (i.e. the rows are the  generation number and the columns are 4 quartiles of ram size). 
df['bluetooth_bin'] = (df['bluetooth'] == 'Yes').astype(int)
df_has_bt = df.pivot_table(index='gen' , columns=ram,values = 'bluetooth_bin', aggfunc = sum)
df_total = df.pivot_table(index='gen',columns=ram,values='bluetooth_bin',aggfunc='count')
df_has_bt/df_total

ram,"(0, 1000]","(1000, 2000]","(2000, 3000]","(3000, 4000]"
gen,,,,
2,0.510204,0.526316,0.508197,0.540323
3,0.478723,0.488000,0.418440,0.441667
4,0.475728,0.465909,0.511945,0.542857


In [11]:
df['bluetooth_bin']

0       0
1       1
2       1
3       1
4       1
       ..
1995    1
1996    1
1997    0
1998    0
1999    1
Name: bluetooth_bin, Length: 2000, dtype: int32